<a href="https://colab.research.google.com/github/woranov/amlta-project/blob/main/notebooks/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    import amlta  # noqa F401
except ImportError:
    %pip install --no-dependencies git+https://github.com/woranov/amlta-project.git

In [ ]:
from pathlib import Path

import yaml
from tqdm.auto import tqdm

try:
    from google.colab import drive  # pyright: ignore[reportMissingImports]

    IN_COLAB = True
except ImportError:
    IN_COLAB = False

from amlta.config import config
from amlta.probas import processes

In [3]:
if IN_COLAB:
    mount_point = Path("/content/drive")
    drive_path = mount_point / "MyDrive"

    # edit
    data_dir = drive_path / "uni" / "ws2425" / "amlta" / "project" / "data"

    config.update(data_dir=data_dir)

    if not mount_point.exists():
        drive.mount(str(mount_point))

In [4]:
# by default lci result only
process_uuids = processes.read_uuids()

In [ ]:
useless_description = "Kurzinfo: Datensatz aus GEMIS. Negative Werte durch Gutschriftenrechnung. \n \nGEMIS steht f\u00fcr \u201cGlobales Emissions-Modell Integrierter Systeme\u201c; es ist ein Softwaretool des \u00d6ko-Instituts. GEMIS wurde 1987 erstmals angewendet und wird seitdem weiterentwickelt. \n \nDie GEMIS-Datens\u00e4tze beruhen - je nach Anwendung - auf unterschiedlichen Methoden; auch der zeitliche und der \u00f6rtliche Bezug der Datens\u00e4tze sind verschieden.\n \nMethode bei Prozessen mit mehreren Outputs:\n \nZur Modellierung der Datens\u00e4tze zu Multi-Output Prozessen wird in GEMIS die Methode der Systemerweiterung verwendet. Hierbei werden Datens\u00e4tze, in denen jeweils alle Inputs, alle Outputs und alle Umweltaspekte eines Multi-Output Prozesses ausgewiesen sind, als \u201cBrutto\u201c bezeichnet. Durch Subtraktion von \u201aBonus\u2019-Prozessen, die jeweils einen der Outputs auf herk\u00f6mmliche Weise bereitstellen, entsteht ein Nettoprozess, in denen das substituierte Nebenprodukt als Gutschrift erscheint. Die Gutschrift ist dabei kein realer Output des Prozesses, sondern ein rechnerischer \u201aMerker\u2019. \n \nBeispiel: \n \nMulti-Output Prozess Biogas-BZ-MC-HKW-D-2020/brutto: Output ist 1 TJ Elektrizit\u00e4t und 0,6 TJ W\u00e4rme, der \u201cNetto\u201c-Datensatz soll sich aber nur auf die Elektrizit\u00e4t beziehen. Durch Subtraktion des Bonusprozesses W\u00e4rme-Bonus-Gas-Hzg-D-2020 mit dem Output W\u00e4rme(0,6 TJ) entsteht der \u201cNetto\u201c-Datensatz Biogas-BZ-MC-HKW-D-2020/Gas, f\u00fcr den als Output 1 TJ Elektrizit\u00e4t und 0,6 TJ \u201aGutschrift W\u00e4rme-Bonus-f\u00fcr-KWK (Bio)-2020 bei W\u00e4rme-Bonus-Gas-Hzg-D-2020\u2019 angegeben werden; die Gutschrift stellt keinen Stoff- oder Energiefluss des Prozesses dar, sie ist allein rechnerisch begr\u00fcndet.\n \n\n \nTransport:\n \nAngaben zu den angesetzten Transportdistanzen werden nicht gegeben.\n \nAbschneidekriterien:\n \nWasser wird in der Regel nur auf der Inputseite angegeben (etwa als K\u00fchlwasser), auch wenn es den Prozess wieder verl\u00e4sst als Abwasser.\n Weitere Angaben zu angewendeten Abschneidekriterien werden nicht gegeben.\n \nBesondere Nomenklatur:\n \nZahlreiche Abk\u00fcrzungen f\u00fcr Brennstoffe aus Biomasse und entsprechende Technologien.\n \nBesonderheiten auf Datensatzebene:\n \nDie Datens\u00e4tze sind mit Vorketten-Datens\u00e4tzen verkn\u00fcpft, in denen die jeweils ben\u00f6tigten Vorprodukte, Energien und Transportleistungen erzeugt werden. Die Daten zu den Umweltaspekten werden erstens \u201cdirekt\u201c (d.h., nur aus dem jeweiligen Prozess, falls dieser direkt zu Umweltaspekten beitr\u00e4gt) als auch \u201cmit Vorkette\u201c (d.h., einschlie\u00dflich aller vorausgehenden Prozesse) ausgewiesen. \n Negative Werte f\u00fcr Stofffl\u00fcsse kommen in GEMIS regelm\u00e4\u00dfig vor; sie entstehen durch die Anwendung von Systemerweiterung um Multi-Output Prozesse in Single Output Prozesse umzurechnen. \n Teilweise werden Aufwendungen f\u00fcr Produktionsmittel (Anlagen, Fahrzeuge etc.) aufgef\u00fchrt (als Stofffl\u00fcsse im Input); diese sind jedoch nicht auf die funktionelle Einheit bezogen, sondern werden als absolute Werte angegeben; sie werden nur als Input und nicht als Output (Entsorgung der Betriebsmittel) angegeben. \n Die durch die Herstellung dieser Produktionsmittel verursachten Umweltaspekte sind dagegen \u00fcber Leistung, j\u00e4hrliche Auslastung und Lebensdauer auf die funktionelle Einheit bezogen \n \nWeiterf\u00fchrende Hinweise und Literatur:\n \n#1: Fritsche, U.R., Schmidt, K.: Globales Emissions-Modell Integrierter Systeme (GEMIS), Version 4.2, Handbuch, Darmstadt, August 2004.\n #2: Fritsche, U.R., Schmidt, K.: Globales Emissions-Modell Integrierter Systeme (GEMIS), Version 4.1, Handbuch, Darmstadt, Darmstadt, Januar 2003.\n #3: Fritsche, U., et al.: Stoffstromanalyse zur nachhaltigen energetischen Nutzung von Biomasse, Verbundprojekt gef\u00f6rdert vom BMU im Rahmen des ZIP, Projekttr\u00e4ger: FZ J\u00fclich, Mai 2004, Anhangband zum Endbericht.\n #4: Fritsche, U., et al.: Umweltanalyse von Energie-, Transport- und Stoffsystemen: Gesamt-Emissions-Modell integrierter Systeme (GEMIS) Version 2.1 - erweiterter und aktualisierter Endbericht, U. Fritsche u.a., i.A. des Hessischen Ministeriums f\u00fcr Umwelt, Energie und Bundesangelegenheiten (HMUEB), ver\u00f6ffentlicht durch HMUEB, Wiesbaden 1995\n \n\n\n"


def process_to_yaml(process: processes.ProcessData):
    name = process.processInformation.dataSetInformation.name.baseName.get() or ""
    synonyms = process.processInformation.dataSetInformation.synonyms
    if synonyms:
        name += f" ({synonyms.get()})"

    year = process.processInformation.time.referenceYear
    geography = process.processInformation.geography.locationOfOperationSupplyOrProduction.location

    classes = []
    if process.processInformation.dataSetInformation.classificationInformation.classification:
        for classification in process.processInformation.dataSetInformation.classificationInformation.classification:
            classes.append(" / ".join(class_.value for class_ in classification.class_))

    # uuid = process.processInformation.dataSetInformation.UUID

    description = process.processInformation.dataSetInformation.generalComment.get()
    if description == useless_description:
        description = ""

    return yaml.dump(
        {
            "name": name,
            "year": year,
            "description": description,
            "geography": geography,
            "classes": classes,
        },
        indent=2,
        sort_keys=False,
        width=1000,
    )


def save_yamls(uuids: list[str]):
    directory = config.ilcd_dir / "processes_yaml"
    directory.mkdir(exist_ok=True)

    process_iter = processes.ProcessData.from_uuids(uuids)
    for process in tqdm(process_iter, total=len(uuids)):
        yaml_str = process_to_yaml(process)
        file = directory / f"{process.processInformation.dataSetInformation.UUID}.yaml"
        if file.exists():
            continue
        with open(file, "w") as f:
            f.write(yaml_str)

In [42]:
save_yamls(process_uuids)

  0%|          | 0/11762 [00:00<?, ?it/s]